# Project 3: Where's Waldo?

Due Mar. 4th

So far, we've mostly focused on using imagery to do stuff for which it is better suited than a human: calculating camera locations from imagery, finding an optimal projective transform to stitch images together, and (soon) we'll be doing "structure from motion" in which we create 3D models of the world from collections of 2D images.  These are tasks primarily based around measuring things and doing calculations.  On the other side of the coin is object recognition (identifying the semantic content of a scene), and the best contemporary computer vision algorithms do object recognition at roughly the level of a 2 year old human (with some exceptions).  For this (mini-)project, we're going to delve into a topic that sort of straddles the line between these two general realms of computer vision.

As a motivating example, did you ever play the game Where's Waldo.  There are books filled with images like the following:
<img src='waldo_1.jpg'>
The objective, of course, is to find Waldo, the man in the red striped shirt and beanie wearing glasses.  He looks like this:
<img src='waldo_template.jpg'>
These scenes are (obviously) intended to have a bunch of visual clutter to make this task reasonably challenging.

Your task will be to come up with an algorithm that locates the template image (Waldo's face) and the target image (the larger scene).  This is called *template matching*, and it's a primitive form of feature recognition.  

## Implementation
### Template Matching
Template matching works in a way that is very similar to filtering:  slide the template image over every location in the target image, computing some sort of metric at each position.  In practice, one commonly used choice for an error metric is the one that you've already used for matching keypoint descriptors: z-normalized sum square error.  Another choice is [normalized cross-correlation](https://en.wikipedia.org/wiki/Cross-correlation#Normalization).  Once these metrics have been computed, simply find the argmin (for SSE) or argmax (for NCC), and this will be the location of the best match.  

**Your task is to implement template matching.  Use 'waldo_template.jpg' as the template and 'waldo_1.jpg' as the target image.  Where's Waldo? **

### Not so fast!!!  What about scale!
Oh, no.  As it turns out, the template I've provided is not the same scale as the Waldo in the image.  To deal with this, you'll need to create an image pyramid for the template (See Szeliski 3.5, and [Mubarak Shah's lecture on this topic](https://www.youtube.com/watch?v=KO7jJt0WHag&feature=youtu.be)).  This essentially just means creating a sequence of downsampled images of the template, and trying each one in hopes that one of the resulting down-scaled templates matches the feature in the target image.  **Create a sequence of templates with which to perform feature matching, each one 1/2 the resolution of the previous (so 1/4 the total number of pixels).  To avoid aliasing, before downsampling perform a $\sigma=1$ Gaussian Blur of the image.  Once you've built your image pyramid, find the argmin/max in 3 dimensions (u,v,template scale)**.

## Generalization
**Waldo appears in every Where's Waldo image (obviously).  Try using the same technique on 'waldo_2.jpg'.  Does the algorithm work?**  I confess that I pulled the image of waldo for the template directly from 'waldo_1.jpg', so for the correct scale, there is something close to an exact match (i.e. SSE=0).  However, Waldo, while easily recognizable to the human eye after undergoing the small scale deformations associated with artistic license, is not so easily recognizable via template matching.  We will return to a similar problem when discussing object recognition, and hopefully this example will motivate the need to come up with representations of objects (like Waldo) that are more robust.


In [ ]:
import numpy as np
import matplotlib as plt

#Finds the closest match from a template image (T_img) to the main image (F_img)
def template_match(T_img, F_img):
    #The number of pixels in T_img
    n = T_img.shape[0] * T_img.shape[1]
    
    #The radius of the Y Axis in T_img
    T_y_radius = T_img.shape[0] // 2
    
    #The radius of the X Axis in T_img
    T_x_radius = T_img.shape[1] // 2
    
    #The standard deviation of T_img
    T_std = T_img.std( axis = 1 )
    
    #Intialized list to hold the sums to later find the argmax of
    sum_list = np.zeros(F_img.shape[0], F_img.shape[1])
    
    for i in range(T_y_radius, len(F_img) - T_y_radius)):
        for j in range(T_x_radius, len(F_img[0]) - T_x_radius):
            #Makes a sub image of F_img based on the size of the T_img
            F_subImg = F_img[ i - T_y_radius : i + T_y_radius + 1, j - T_x_radius : j + T_x_radius + 1]
            #The standard deviation of the sub image
            F_std = F_subImg.std(axis=1)
            #Finds the Normilized Cross-Correlation
            curr_sum = ( 1 / n ) * ( ( 1 / ( F_std * T_std ) ) * np.matmul( F_subImg, T_img ) ).sum()
            #Store the sum at the appropriate index corresponding to the F_img
            sum_list[j,i] = curr_sum
    
    #Finds the argmax of the list of sums
    maxIndex = np.argmax(sum_list, axis=1)
    
    return maxIndex
        